Rita Kurban

Professor Shekhar

CS152

In [1]:
from copy import deepcopy

In [2]:
class Literal:
    ''' Define a Literal with Name, Sign, and Value attributes.'''
    def __init__(self, name, sign = True):
        self.name = name
        self.sign = sign
        self.value = None
        
    def __repr__(self):
        ''' Print the literal with its sign.''' 
        if self.sign:
            return self.name
        return "-" + self.name
        
    def __neg__(self):
        ''' Overload the negation.''' 
        neg = Literal(self.name)
        neg.sign = False
        return neg
        
    def __eq__(self, other):
        ''' Make equality independent of sign.''' 
        return self.name == other.name
    
    def __hash__(self):
        ''' Hash function to work with sets and dictionaries.''' 
        return hash(str(self.sign) + self.name)

In [3]:
#Extension 1

def degree(clauses, symbols): 
    """ Returns the most frequent literal in the clauses."""    
    lis = dict()
    max_degree = None
    count = 0
    
    # Go through every literal and determine its degree
    for clause in clauses:
        for symbol in clause:
            abs_symbol = deepcopy(symbol)
            abs_symbol.sign = True
            if abs_symbol not in symbols:
                continue
            if abs_symbol not in lis:
                lis[abs_symbol] = 1
            else:
                lis[abs_symbol] += 1

            # Update the count to the literal with the highest degree so far    
            if lis[abs_symbol] > count:
                count = lis[abs_symbol]
                max_degree = abs_symbol
    # Return the most frequent symbol
    return max_degree

In [4]:
#Extension 2
def pure_symbol(clauses, symbols):    
    """ Returns the literal appearing with the same sign in all the clauses of the given KB."""  
    # Go through all the literals in every clause and separate them into positive and negative
    pos = set({})
    neg = set({})
    for clause in clauses:
        for symbol in clause:
            # Check if symbol is in symbols
            abs_symb = deepcopy(symbol)
            abs_symb.sign = True
            if abs_symb not in symbols:
                continue
            if symbol.sign:
                if symbol not in pos:
                    pos.add(abs_symb)
            else:
                if symbol not in neg:
                    neg.add(abs_symb)
    # Determine pure symbols                
    pure = []
    pure = [s for s in pos if s not in neg] + [s for s in neg if s not in pos]
    # Use the degree heuristic to determine the most frequent pure symbol
    max_degree = degree(clauses, pure)
    # Return the literal and its value
    if max_degree in pos:
        return max_degree, True
    else:
        return max_degree, False

In [5]:
def unit_clause(clauses,symbols):
    """ Finds clauses with only one literal."""
    for clause in clauses:
        if len(clause) != 1:
            continue
        for symbol in clause:
            abs_symb = deepcopy(symbol)
            abs_symb.sign = True
            if abs_symb in symbols:
                return abs_symb, symbol.sign
    return None, False

In [6]:
def check_clauses(clauses, model):
    ''' Function that takes a set of clauses and checks whether all of them are satisfiable.
    Returns True if all clauses are True, False if at least one of them is False, and "Continue"
    if some of the clauses' symbols aren't in the model.'''  
    correct = 0
    for clause in clauses:
        for symbol in clause:
            s = deepcopy(symbol)
            s.sign = True
            # Check if symbol is in the model
            if s not in model:
                return "Continue"
            if symbol.sign == True:
                symbol.value = model[s]
            else:
                symbol.value = not model[s]
            # If at least one symbol is True, the clause is True
            if symbol.value == True:
                correct +=1
                break
    # If the number of correct clauses is the same as their number, return True
    if correct == len(clauses):
        return True
    # If there are no missing symbols and not all clauses are true, return False
    else:
        return False

In [7]:
def DPLL(clauses, symbols, model):
    ''' Am implementation of the DPLL algorithm with pure symbol,
    unit, and degree heuristics.'''
    # Check if any of the clauses are False
    if check_clauses(clauses, model) == False:
        return False, None
    # Check if all clauses are True
    if check_clauses(clauses, model) == True:
        # If the clause is True, the rest of the symbols can be "Free".
        for symbol in symbols:
            model[symbol] = "Free"
        # Return True and model
        return True, model
    # Pure symbol heuristic
    pure, value = pure_symbol(clauses, symbols)
    if pure != None:
        current = pure
        symbols.remove(pure)
        model[current] = value
        r, m = DPLL(clauses, symbols, model)
        if r == True:
            return True, m
    # Unit clause heuristic
    unit, val = unit_clause(clauses, symbols)
    if unit != None:
        current = unit
        symbols.remove(unit)
        model[current] = val
        r0, m0 = DPLL(clauses, symbols, model)
        if r0 == True:
            return True, m0
    # If both heuristics didn't work, select the symbol that appears in the KB the most
    high_degree = degree(clauses,symbols)
    current = high_degree
    symbols.remove(high_degree)
    model[current] = False
    r1, m1 = DPLL(clauses, symbols, model)
    if r1 == True:
        return True, m1
    model[current] = True
    r2, m2 = DPLL(clauses, symbols, model)
    if r2 == True:
        return True, m2
    # If none of the results is True, return False
    return False, None


In [8]:
def DPLL_satisfiable(s):
    """ Function that transforms KB into a set of clauses and symbols and calls DPLL()."""
    # Create a list of clauses
    clauses = [clause for clause in s]
    # Create a set of symbols
    symbols = set()
    for clause in clauses:
        for symbol in clause:
            symb = deepcopy(symbol)
            symb.sign = True
            symbols.add(symb)
    return DPLL(clauses, symbols, {})

In [9]:
# Excercise 7.20 from Russel and Norvig

# Define Literals
A = Literal('A')
B = Literal('B')
C = Literal('C')
D = Literal('D')
E = Literal('E')
F = Literal('F')

KB = [{-A, B, E}, {-B, A}, {-E, A}, {-E, D}, 
      {-C, -F, -B}, {-E, B}, {-B, F}, {-B, C}]

DPLL_satisfiable(KB)

(True, {C: False, E: False, B: False, F: False, D: True, A: False})